In [1]:
import os
import json
import numpy as np
import pandas as pd
import operator
from datetime import datetime
import datetime
import warnings

warnings.filterwarnings("ignore")

from sklearn.preprocessing import OneHotEncoder

In [2]:
data_path=os.path.join(os.path.abspath(os.getcwd()),'raw_data')
df=pd.read_csv(os.path.join(data_path,'startups_modified.csv'))

In [3]:
df.shape

(56667, 61)

In [4]:
df.head(10)

,founded_on,num_funding_rounds,last_equity_funding_type,last_equity_funding_total,last_funding_at,headquartersCountry,employeeCount,Advertising,Agriculture and Farming,Clothing and Apparel,...,Hardware,Science and Engineering,Software_y,Sustainability,Round 1,Round 2,Round 3,Round 4,Round 5,Target
0,2016-01-01,1,seed,NaN,2018-05-30,CN,31.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
1,2017-01-01,2,seed,NaN,2022-11-30,US,6.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,50000.0,NaN,NaN,NaN,NaN,0.0
2,2016-07-11,1,seed,500000.0,2016-07-11,US,6.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,500000.0,NaN,NaN,NaN,NaN,0.0
3,2018-02-24,1,pre_seed,NaN,2018-08-01,US,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
4,2016-02-01,2,NaN,NaN,2017-01-20,US,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9600000.0,NaN,NaN,NaN,NaN,0.0
5,2016-04-01,3,series_unknown,176164.0,2022-06-23,CN,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,463384.0,3000277.0,176164.0,NaN,NaN,0.0
6,2018-01-01,1,seed,NaN,2020-05-08,VN,31.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
7,2016-01-01,1,series_unknown,NaN,2016-12-12,CO,6.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
8,2018-02-01,2,seed,NaN,2021-10-01,CA,6.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
9,2018-01-01,1,NaN,NaN,2020-12-21,DK,NaN,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,164650.0,NaN,NaN,NaN,NaN,0.0


In [5]:
print(df[df.last_equity_funding_type.isnull()]['last_equity_funding_type'].shape[0])
df['last_equity_funding_type'].fillna("series_unknown",inplace=True)
print(df[df.last_equity_funding_type.isnull()]['last_equity_funding_type'].shape[0])

8049
0


In [6]:
print(df[df.last_equity_funding_total.isnull()]['last_equity_funding_total'].shape[0])
df['last_equity_funding_total'] = np.where(df['last_equity_funding_total'].isnull(), df['Round 5'],df['last_equity_funding_total'])
print(df[df.last_equity_funding_total.isnull()]['last_equity_funding_total'].shape[0])
df['last_equity_funding_total'] = np.where(df['last_equity_funding_total'].isnull(), df['Round 4'],df['last_equity_funding_total'])
print(df[df.last_equity_funding_total.isnull()]['last_equity_funding_total'].shape[0])
df['last_equity_funding_total'] = np.where(df['last_equity_funding_total'].isnull(), df['Round 3'],df['last_equity_funding_total'])
print(df[df.last_equity_funding_total.isnull()]['last_equity_funding_total'].shape[0])
df['last_equity_funding_total'] = np.where(df['last_equity_funding_total'].isnull(), df['Round 2'],df['last_equity_funding_total'])
print(df[df.last_equity_funding_total.isnull()]['last_equity_funding_total'].shape[0])
df['last_equity_funding_total'] = np.where(df['last_equity_funding_total'].isnull(), df['Round 1'],df['last_equity_funding_total'])
print(df[df.last_equity_funding_total.isnull()]['last_equity_funding_total'].shape[0])

21460
21316
21146
20751
19509
13706


In [7]:
df_equity_funding_type_median=pd.DataFrame(df[df['last_equity_funding_total'].notnull()].groupby(['last_equity_funding_type']).median()['last_equity_funding_total']).reset_index()
df_equity_funding_type_median.rename(columns={'last_equity_funding_total':'last_equity_funding_median'},inplace=True)
df_equity_funding_type_median

,last_equity_funding_type,last_equity_funding_median
0,angel,451277.0
1,corporate_round,3107924.5
2,equity_crowdfunding,386067.5
3,initial_coin_offering,8774710.5
4,post_ipo_equity,10762663.0
5,pre_seed,120000.0
6,private_equity,30000000.0
7,seed,652650.0
8,series_a,7142020.5
9,series_b,24795061.0


In [8]:
df = df.merge(df_equity_funding_type_median, how='left',on='last_equity_funding_type')
df['last_equity_funding_total'] = np.where(df['last_equity_funding_total'].isnull(), df['last_equity_funding_median'],df['last_equity_funding_total'])
df.drop('last_equity_funding_median', axis=1,inplace=True)
print(df[df.last_equity_funding_total.isnull()]['last_equity_funding_total'].shape[0])

1


In [9]:
df[df.last_equity_funding_total.isnull()]['last_equity_funding_total']

50287   NaN
Name: last_equity_funding_total, dtype: float64

In [10]:
index_to_drop=df[df.last_equity_funding_total.isnull()]['last_equity_funding_total'].index[0]

In [11]:
df.drop([index_to_drop],inplace=True)

In [12]:
print(df[df.last_equity_funding_total.isnull()]['last_equity_funding_total'].shape[0])

0


In [27]:
df.last_equity_funding_type.isnull().value_counts()

False    56666
Name: last_equity_funding_type, dtype: int64